In [1]:
import traci
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict


def analyze_random_actions(environment, episodes=5, total_sim_time=3600, action_duration=80):
    """
    Select random actions, apply them for fixed duration, and analyze flow ratios.
    
    Parameters:
        environment: The traffic environment
        episodes: Number of simulation episodes to run
        total_sim_time: Total simulation time per episode (seconds)
        action_duration: Duration to apply each action (seconds)
    """
    # Storage for collected data
    all_flow_ratios = []
    all_actions = []
    all_waiting_times = []
    
    print("\nRandom Action Analysis")
    print("Format: Episode | Action | Flow Ratios | Avg Wait Time")
    print("-" * 75)
    
    for episode in range(episodes):
        # Start the simulation
        environment.start_simulation()
        
        # Data for this episode
        episode_flow_ratios = []
        episode_actions = []
        episode_waiting_times = []
        
        # Track vehicles for metrics
        total_waiting_time = 0.0
        vehicle_counts = set()
        
        # Run simulation steps
        step = 0
        while step < total_sim_time:
            # Select random action
            action = random.choice(list(environment.action_space.keys()))
            episode_actions.append(action)
            
            # Apply the action
            green_durations = environment.action_space[action]
            environment.update_traffic_light_program("J1", green_durations)
            
            # Record initial vehicle counts for flow tracking
            controlled_lanes = traci.trafficlight.getControlledLanes("J1")
            initial_vehicles = {lane: set(traci.lane.getLastStepVehicleIDs(lane)) for lane in controlled_lanes}
            
            # Group lanes by phase (assuming 3 lanes per phase based on your setup)
            phase_lanes = [
                controlled_lanes[0:3],   # Lanes for first phase
                controlled_lanes[3:6],   # Lanes for second phase
                controlled_lanes[6:9],   # Lanes for third phase
                controlled_lanes[9:12]   # Lanes for fourth phase
            ]
            
            # Run simulation steps for the action duration
            action_waiting_time = 0
            for _ in range(action_duration):
                traci.simulationStep()
                
                # Update metrics
                current_wait = sum(environment.compute_waiting_time().values())
                action_waiting_time += current_wait
                total_waiting_time += current_wait
                
                # Track vehicles
                current_vehicles = set(traci.vehicle.getIDList())
                vehicle_counts.update(current_vehicles)
            
            # Calculate average waiting time for this action
            avg_action_wait = action_waiting_time / action_duration
            episode_waiting_times.append(avg_action_wait)
            
            # Calculate flow ratios for each phase
            phase_flow_ratios = []
            
            for phase_idx, lanes in enumerate(phase_lanes):
                phase_in = 0
                phase_out = 0
                
                for lane in lanes:
                    # Get final vehicle set
                    final_vehicles = set(traci.lane.getLastStepVehicleIDs(lane))
                    
                    # Calculate inflow (vehicles in final set that weren't in initial set)
                    inflow = len(final_vehicles - initial_vehicles[lane])
                    
                    # Calculate outflow (vehicles in initial set that aren't in final set)
                    outflow = len(initial_vehicles[lane] - final_vehicles)
                    
                    phase_in += inflow
                    phase_out += outflow
                
                # Calculate flow ratio with epsilon to avoid division by zero
                epsilon = 1e-6
                if phase_in + epsilon > 0:
                    ratio = phase_out / (phase_in + epsilon)
                else:
                    ratio = 0
                    
                phase_flow_ratios.append(ratio)
            
            episode_flow_ratios.append(phase_flow_ratios)
            print(f"Episode {episode+1} | Action {action:2d} | Flow Ratios: {[f'{r:.2f}' for r in phase_flow_ratios]} | Wait: {avg_action_wait:.1f}")
            
            # Update step counter
            step += action_duration
        
        # Calculate episode metrics
        total_vehicles = len(vehicle_counts)
        waiting_time_per_vehicle = total_waiting_time / total_vehicles if total_vehicles > 0 else 0
        
        print(f"\nEpisode {episode+1} Summary:")
        print(f"Total Wait Time: {total_waiting_time:.1f}")
        print(f"Total Vehicles: {total_vehicles}")
        print(f"Wait Time per Vehicle: {waiting_time_per_vehicle:.1f}")
        print("-" * 50)
        
        # Store episode data
        all_actions.append(episode_actions)
        all_flow_ratios.append(episode_flow_ratios)
        all_waiting_times.append(episode_waiting_times)
        
        # Clean up
        environment.close_simulation()
    
    # Analyze the collected data
    analyze_flow_ratio_data(all_actions, all_flow_ratios, all_waiting_times)
    
    return all_actions, all_flow_ratios, all_waiting_times


def analyze_flow_ratio_data(actions, flow_ratios, waiting_times):
    """Analyze flow ratio data to find patterns and correlations."""
    # Flatten the data for analysis
    flat_actions = [action for episode_actions in actions for action in episode_actions]
    flat_flow_ratios = [ratio for episode_ratios in flow_ratios for ratio in episode_ratios]
    flat_waiting_times = [time for episode_times in waiting_times for time in episode_times]
    
    # Calculate average flow ratio for each action
    action_to_flow = defaultdict(list)
    action_to_wait = defaultdict(list)
    
    for action, ratios, wait in zip(flat_actions, flat_flow_ratios, flat_waiting_times):
        # Average flow ratio across all phases
        avg_ratio = sum(ratios) / len(ratios)
        action_to_flow[action].append(avg_ratio)
        action_to_wait[action].append(wait)
    
    # Calculate mean and std for each action
    action_stats = {}
    for action in action_to_flow:
        avg_flow = np.mean(action_to_flow[action])
        std_flow = np.std(action_to_flow[action])
        avg_wait = np.mean(action_to_wait[action])
        std_wait = np.std(action_to_wait[action])
        action_stats[action] = {
            'avg_flow': avg_flow,
            'std_flow': std_flow,
            'avg_wait': avg_wait,
            'std_wait': std_wait,
            'count': len(action_to_flow[action])
        }
    
    # Print action statistics
    print("\nAction Statistics:")
    print("Action | Avg Flow Ratio | Flow Std Dev | Avg Wait Time | Wait Std Dev | Count")
    print("-" * 80)
    
    for action, stats in sorted(action_stats.items()):
        print(f"{action:5d} | {stats['avg_flow']:13.2f} | {stats['std_flow']:11.2f} | {stats['avg_wait']:12.1f} | {stats['std_wait']:11.1f} | {stats['count']:5d}")
    
    # Calculate flow ratio bin distribution
    bins = {'low': 0, 'medium': 0, 'high': 0}
    for episode_ratios in flow_ratios:
        for phase_ratios in episode_ratios:
            for ratio in phase_ratios:
                if ratio < 0.8:
                    bins['low'] += 1
                elif ratio < 1.2:
                    bins['medium'] += 1
                else:
                    bins['high'] += 1
    
    total_ratios = sum(bins.values())
    print("\nFlow Ratio Distribution:")
    print(f"Low (<0.8): {bins['low']} ({bins['low']/total_ratios*100:.1f}%)")
    print(f"Medium (0.8-1.2): {bins['medium']} ({bins['medium']/total_ratios*100:.1f}%)")
    print(f"High (>1.2): {bins['high']} ({bins['high']/total_ratios*100:.1f}%)")
    
    # Calculate correlation between flow ratio and waiting time
    flat_avg_ratios = [sum(ratios)/len(ratios) for ratios in flat_flow_ratios]
    correlation = np.corrcoef(flat_avg_ratios, flat_waiting_times)[0, 1]
    print(f"\nCorrelation between flow ratio and waiting time: {correlation:.3f}")
    
    # Visualize the data
    plt.figure(figsize=(12, 8))
    
    # Plot 1: Flow Ratio Distribution
    plt.subplot(2, 2, 1)
    plt.bar(['Low', 'Medium', 'High'], [bins['low'], bins['medium'], bins['high']])
    plt.title('Flow Ratio Distribution')
    plt.ylabel('Count')
    
    # Plot 2: Average Flow Ratio by Action
    actions_list = sorted(action_stats.keys())
    avg_flows = [action_stats[a]['avg_flow'] for a in actions_list]
    
    plt.subplot(2, 2, 2)
    plt.bar(actions_list, avg_flows)
    plt.title('Average Flow Ratio by Action')
    plt.xlabel('Action')
    plt.ylabel('Avg Flow Ratio')
    
    # Plot 3: Average Wait Time by Action
    avg_waits = [action_stats[a]['avg_wait'] for a in actions_list]
    
    plt.subplot(2, 2, 3)
    plt.bar(actions_list, avg_waits)
    plt.title('Average Wait Time by Action')
    plt.xlabel('Action')
    plt.ylabel('Avg Wait Time')
    
    # Plot 4: Flow Ratio vs Wait Time Scatter
    plt.subplot(2, 2, 4)
    plt.scatter(flat_avg_ratios, flat_waiting_times, alpha=0.5)
    plt.title(f'Flow Ratio vs Wait Time (Corr: {correlation:.3f})')
    plt.xlabel('Average Flow Ratio')
    plt.ylabel('Wait Time')
    
    plt.tight_layout()
    plt.savefig('flow_ratio_analysis.png')
    plt.close()
    

if __name__ == "__main__":
    from environment import Environment
    
    # Define configuration (use your actual config paths)
    sumo_binary = "sumo-gui"  # or "sumo" for non-GUI mode
    sumo_config = r"C:\Users\Administrator\Desktop\MySumo\Stage 10\SUMO files\stage10.sumocfg"
    
    # Create environment instance
    env = Environment(sumo_config, sumo_binary)
    
    # Run analysis
    actions, flow_ratios, waiting_times = analyze_random_actions(
        environment=env,
        episodes=5,
        total_sim_time=3600,
        action_duration=80
    )
    
    print("\nAnalysis complete! Results saved to flow_ratio_analysis.png")


Random Action Analysis
Format: Episode | Action | Flow Ratios | Avg Wait Time
---------------------------------------------------------------------------
Episode 1 | Action  7 | Flow Ratios: ['0.00', '0.00', '0.00', '0.00'] | Wait: 0.8
Episode 1 | Action  7 | Flow Ratios: ['0.41', '0.08', '0.06', '0.00'] | Wait: 17.6
Episode 1 | Action 21 | Flow Ratios: ['0.13', '0.71', '0.12', '0.29'] | Wait: 733.9
Episode 1 | Action  2 | Flow Ratios: ['0.76', '0.25', '0.45', '0.33'] | Wait: 1264.5
Episode 1 | Action 11 | Flow Ratios: ['0.69', '0.36', '0.12', '0.60'] | Wait: 2680.2
Episode 1 | Action 18 | Flow Ratios: ['0.47', '0.25', '0.35', '0.29'] | Wait: 3481.3
Episode 1 | Action 29 | Flow Ratios: ['0.33', '0.18', '0.65', '0.33'] | Wait: 4110.5
Episode 1 | Action 33 | Flow Ratios: ['0.28', '0.56', '0.23', '0.62'] | Wait: 5183.6
Episode 1 | Action  9 | Flow Ratios: ['0.29', '0.86', '0.25', '1.00'] | Wait: 5852.8
Episode 1 | Action  6 | Flow Ratios: ['0.33', '0.83', '3.00', '0.13'] | Wait: 6302.1
E

In [ ]:
before_vehicles = {}
for i in range(4):
    before_vehicles[i] = set(i+j for j in range(4))
print(before_vehicles)

: 